# **Notebook de transfert des donnees du site nba.com depuis le site ou un fichier intermediaire**

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import pandas as pd
import sys, os, re, time # c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'D:\basket\code\Basket\src')
import Connexion_Transfert as ct

from datetime import datetime
from TelechargementDonnees import JoueursSiteNba, DriverFirefox, gererCookieNba, gererCookieTtfl, fermerIncrustationNba, simplifierNomJoueur, JourneeSiteNba, Blessures, JoueursChoisisTtfl, Calendrier
from TeleversementJourneeBdd import JourneeBdd, insererBlessesInconnusMatchBefore, miseAJourBlessesBdd, insererBlessesInconnusPasMatchBefore, transfertCalendrier, televerserJourneeSiteNba, PasDeMatchError
from Ihm.Initialiser_donnees_IHM import lastDates
from sqlalchemy import MetaData
from datetime import timedelta, date
from ParamsWeb import divTestCalendrier, refElementNom, refDivCarac, refParagrapheCarac

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException
from selenium.webdriver.support.ui import Select
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

In [4]:
import Ihm.MainWindow as mw

In [5]:
pd.set_option('display.max_rows', 500)

# 1. Données de joueurs 
Il s'agit de transfere dans le cas des donnees de joueuer telechargees en amont, ou d'ajouter des joueurs non presents au moment d'insertion de match

>## 1.1 Enumerations des types de position
A creer a la main

In [ ]:
with ct.ConnexionBdd('basket','maison') as c : 
    pd.DataFrame({'id_position_terrain':['F', 'G', 'C', 'C-F', 'F-G', 'G-F', 'F-C','NC'], 
              'nom_position_terrain':['Forward ; 3 ; ailier', 'Guard ; 1 ou 2  ; meneur ou arriere', 
                                      'Center ; 5 ; Pivot' ,'Power Forward ; 4 ; ailier Fort', 
                                      'Shooting Guard ; 2 ; arriere', 'Shooting Guard ; 2 ; arriere',
                                      'Power Forward ; 4 ; ailier Fort', 'Non connue']}).to_sql(
        'enum_position_terrain', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

> ## 1.2 Donnees chargees depuis la page internet
Là ce n'est que le premier import, on ne gère pas la mise à jour

In [ ]:
#creation de la classe si import depuis le net
#pagejoueuer=JoueursSiteNba(urlPageJoueurs='https://www.nba.com/player/1630173/precious-achiuwa',typeExport='One' ) #joueur unique
pagejoueuer=JoueursSiteNba() #joueurs multiples

In [ ]:
pagejoueuer.listLinkJoueurs

In [ ]:
pagejoueuer.creerDfJoueur()

In [ ]:
driver=DriverFirefox('mano').driver

In [ ]:
refDivCarac='PlayerSummary_playerInfo__1L8sx'
refParagrapheCarac='PlayerSummary_playerInfoValue__mSfou'
refElementNom='flex flex-col text-white'
dicoCaracJoueur={}
driver.get('https://www.nba.com/player/1630173/precious-achiuwa')
time.sleep(3)
#trouver toute les caracteristiques (car les nom sde classe sont les mêmes pour tous)
elements=WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, f"//div[@class='{refDivCarac}']/p[@class='{refParagrapheCarac}']")))
#self.driver.find_elements_by_xpath(f"//div[@class='{refDivCarac}']/p[@class='{refParagrapheCarac}']")
nomPosition=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, f"//div[@class='{refElementNom}']")))
#self.driver.find_element_by_xpath(f"//div[@class='{refElementNom}']")
listeCaracNomPosition=[e.text for e in nomPosition.find_elements_by_xpath(".//*")]
dicoCaracJoueur['taille']=[float(re.sub('(\(|\)|m)','',re.search('(\([1-2]\.[0-9]{2}m\))',e.text).group(1))) for i,e in enumerate(elements) if i==0][0]
dicoCaracJoueur['poids']=[float(re.sub('(\(|\)|kg)','',re.search('(\([0-9]{1,3}kg\))',e.text).group(1))) for i,e in enumerate(elements) if i==1][0]
dicoCaracJoueur['date_entree_nba']=pd.to_datetime(f"{[2021-int(e.text.split()[0]) if e.text != 'Rookie' else 2021 for i,e in enumerate(elements) if i==7][0]  }-10-01")
dicoCaracJoueur['date_naissance']=pd.to_datetime([e.text for e in elements if re.match('[a-z]{0,12} [0-9]{1,2}, [0-9]{4}',e.text.lower())][0])
dicoCaracJoueur['nom']=' '.join(listeCaracNomPosition[-2:])
dicoCaracJoueur['nom_simple']=simplifierNomJoueur(' '.join(listeCaracNomPosition[-2:]))
dicoCaracJoueur['id_position_terrain']='-'.join([e[0] if e in ('Center','Guard', 'Forward') else 'NC' for e in listeCaracNomPosition[0].split(' | ')[-1].split('-')])

In [ ]:
dicoCaracJoueur

In [ ]:
pagejoueuer.dfJoueurs

In [ ]:
with ct.ConnexionBdd('basket','maison') as c : 
    #remmplacer pagejoueuer.dfJoueurs par Joueur si import Json
    Joueur[['nom','id_position_terrain','taille', 'poids','date_entree_nba','nom_simple']].to_sql(
        'joueur', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 2. Données d'équipe
On utilise les données de joueurs pour faire la liste des equipes, et on complete avec les divisions et conférence, puis on transfere

In [ ]:
dicoNom={'MIA':'Miami Heat','MIL':'Milwaukee Bucks','NOP':'New-Orleans Pelicans','SAS':'San-Antonio Spurs',
'PHX':'Phoenix','MEM':'Memphis Grizzlies',    'BKN':'Brooklyn Nets','ORL':'Orlando Magic','LAL':'LosAngeles Lakers','POR':'Portland TrailBlazzers','TOR':'Toronto Raptors','CHI':'Chicago Bulls',
'OKC':'Oklahoma City Thunder','WAS':'Washington Wizards','UTA':'Utah Jazz','SAC':'Sacramento Kings','CHA':'Charlotte Hornets','NYK':'New-York Knicks','DEN':'Denver Nuggets',
'LAC':'Los Angeles Clippers','GSW':'Golden State Warriors','MIN':'Minessota TimberWolves','DET':'Detroit Pistons','DAL':'Dallas Mavericks',
'IND':'Indiana Pacers','ATL':'Atlanta Hawks','CLE':'Cleveland Cavaliers','PHI':'Philadelphia Sixers','BOS':'Boston Celtics','HOU':'Houston Rockets'}
list_equipe=dicoNom.keys()

In [ ]:
dicoDivision={'Central Division': ['MIL', 'CHI', 'IND', 'CLE', 'DET'],'Atlantic Division': ['BOS', 'BKN', 'NYK', 'TOR', 
'PHI'],'Southeast Division' : ['MIA', 'ORL', 'WAS', 'CHA', 'ATL'],'Southwest Division': ['SAS', 'HOU', 'DAL', 'NOP', 
'MEM'],'Northwest Division' : ['DEN', 'MIN', 'OKC', 'POR', 'UTA'],'Pacific Division' : ['LAC', 'LAL', 'PHX', 'GSW', 'SAC']}
listDivision=[k for e in list_equipe for k,v in dicoDivision.items()  if e in v] 

In [ ]:
dicoConference={'Ouest' : ['SAS', 'HOU', 'DAL', 'NOP', 'MEM','DEN', 'MIN', 'OKC', 'POR', 'UTA','LAC', 'LAL', 'PHX', 
    'GSW', 'SAC'],'Est':['MIL', 'CHI', 'IND', 'CLE', 'DET','BOS', 'BKN', 'NYK', 'TOR', 'PHI','MIA', 
                         'ORL', 'WAS', 'CHA', 'ATL']}

listConference=[k for e in list_equipe for k,v in dicoConference.items()  if e in v] 
listNom=[v for k,v in dicoNom.items() for e in list_equipe if k==e] 

In [ ]:
with ct.ConnexionBdd('basket','boulot') as c : 
    pd.DataFrame({'id_equipe':list_equipe,'nom_equipe': listNom,
              'conference':listConference, 'division':listDivision}).to_sql(
        'equipe', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 3. Données de match
Lecture et transfert des données du site vers la bdd

In [18]:
listeDate = [d.strftime('%Y-%m-%d') for d in pd.date_range(start='2023-02-24', periods=15)]
for e, j in enumerate(listeDate):
    print(j)
    try:
        journee = JourneeBdd(j)
        journee.creerAttributsGlobaux()
        journee.exporterVersBdd()
    except PasDeMatchError:
        continue

2023-02-24
0 https://www.nba.com/game/nyk-vs-was-0022200895/box-score#box-score
1 https://www.nba.com/game/cle-vs-atl-0022200896/box-score#box-score
2 https://www.nba.com/game/mia-vs-mil-0022200897/box-score#box-score
3 https://www.nba.com/game/cha-vs-min-0022200898/box-score#box-score
4 https://www.nba.com/game/bkn-vs-chi-0022200900/box-score#box-score
5 https://www.nba.com/game/okc-vs-phx-0022200899/box-score#box-score
6 https://www.nba.com/game/hou-vs-gsw-0022200901/box-score#box-score
7 https://www.nba.com/game/sac-vs-lac-0022200902/box-score#box-score
2023-02-25
0 https://www.nba.com/game/tor-vs-det-0022200903/box-score#box-score
1 https://www.nba.com/game/mia-vs-cha-0022200904/box-score#box-score
2 https://www.nba.com/game/ind-vs-orl-0022200905/box-score#box-score
3 https://www.nba.com/game/nop-vs-nyk-0022200906/box-score#box-score
4 https://www.nba.com/game/den-vs-mem-0022200907/box-score#box-score
5 https://www.nba.com/game/bos-vs-phi-0022200908/box-score#box-score
6 https://ww

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:133:16


# 4. exemple de blessure
le joueur Markelle Fultz s'est fait une rupture des ligaments croisé, exemple d'insertion dans la table blessure

In [ ]:
nomJoueurBlesse='Markelle Fultz'
idTypeBlessure=1
dateBlessure='2021-06-01'
requeteBlessure = f"""insert into donnees_source.blessure(id_joueur,id_type_blessure,date_blessure) 
             select id_joueur,{idTypeBlessure},{dateBlessure} from donnees_source.joueur where lower(nom)=lower{nomJoueurBlesse}"""
with ct.ConnexionBdd('basket','maison') as c : 
    c.execute(c.sqlAlchemyConn,requeteBlessure)

In [ ]:
#dfsInjuries=Blessures().dfInjuries
with ct.ConnexionBdd('basket','maison') as c : 
    #miseAJourBlessesBdd(dfsInjuries,c.sqlAlchemyConn)
    #insererBlessesInconnusMatchBefore(dfsInjuries,c.sqlAlchemyConn)
    insererBlessesInconnusPasMatchBefore(dfsInjuries,c.sqlAlchemyConn)

# 5. TTFL : inserer les joueurs choisis
Inserer dans la base les joueurs deja choisis en ttfl ;  
Pour le moment je télécharge manuellement les donnees depuis trashtalk
à terme on pourra telecaharger auto (attention, gestion mot de passe) ou remplir directement depuis une liste exterieure

In [3]:
JoueurChoisis=JoueursChoisisTtfl(2)

connexion a etablir


In [4]:
JoueurChoisis.transfertBdd()

# 6. tranferer le calendrier des matchs a venir
en fonction d'une date et duree.  
il reste a mettre en place un systeme qui checke en amont les donnees deja presente et delete celle qui ne sont plus dans la journee (ou reportee, a verifier), et qui insere de nouvelle

In [ ]:
transfertCalendrier(2,'2022-03-28', 12)

In [ ]:
cal=Calendrier(2, '23/01/2022', 1)

In [17]:
driver=DriverFirefox('mano')

In [18]:
ddriver=driver.driver

In [19]:
ddriver.get('https://www.nba.com/games?date=2022-02-18')

In [20]:
gererCookieNba(ddriver)

In [13]:
elementsMatch=WebDriverWait(ddriver, 20).until(EC.presence_of_all_elements_located(
                (By.XPATH,"//a[@class='flex-1 px-2 pt-5 h-full block hover:no-underline relative text-sm pt-5 pb-4 mb-1 px-2']")))

In [7]:
if ddriver.find_element_by_xpath("//div[@class='ab-page-blocker']"):
    boutonCloseIncrustation = ddriver.find_element_by_xpath("//button[@class='ab-close-button']")
    boutonCloseIncrustation.click()

In [20]:
while not WebDriverWait(ddriver, 10).until(EC.presence_of_element_located((By.XPATH, divTestCalendrier))):
    try :
        if ddriver.find_element_by_xpath("//div[@class='ab-page-blocker']"):
            boutonCloseIncrustation = ddriver.find_element_by_xpath("//button[@class='ab-close-button']")
            boutonCloseIncrustation.click()
    except TimeoutException :
        pass
    boutonFermetureFenetreConnexion = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,boutonCloseConnexionNba)))
    boutonFermetureFenetreConnexion.click()
    time.sleep(10)

In [22]:
fermerIncrustationNba(ddriver)

In [20]:
boutonFermetureFenetreConnexion = WebDriverWait(ddriver, 5).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='mvpd-modal-close-button']")))

In [24]:
WebDriverWait(ddriver, 10).until(EC.presence_of_element_located((By.XPATH,"// *[contains(text(),\'No scheduled games')]")))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="ab448af3-e9e2-4cab-b721-e0f1e8f9eaa6", element="3d05ae12-eefe-479b-bedb-9af7f22b861a")>

# 7.Test

In [14]:
listeDeroulanteNbPage = 'Page Number Selection Drown Down List'
typeExport = 'All'
classeTableauColonne1 = 'primary text RosterRow_primaryCol__19xPQ'
classeLienNom = 'flex items-center t6 Anchor_complexLink__2NtkO'

In [24]:
driver = DriverFirefox('mano').driver
urlPageJoueurs = 'https://www.nba.com/player/1630173/precious-achiuwa' # 'https://www.nba.com/stats/player/76003/career'
driver.get(urlPageJoueurs)
gererCookieNba(driver)

In [25]:
dicoCaracJoueur = {}
elements = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((
                By.XPATH, f"//div[@class='{refDivCarac}']/p[@class='{refParagrapheCarac}']")))
nomPosition = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, f"//div[@class='{refElementNom}']")))
listeCaracNomPosition = [e.text for e in nomPosition.find_elements(by=By.XPATH, value=".//*")]
textTaille = elements[0].text
textPoids = elements[1].text
textPays = elements[2].text
if len(elements) == 11:
    textDateNaissance = elements[4].text
    textDraft = elements[5].text
elif len(elements) == 12:
    textDateNaissance = elements[5].text
    textDraft = elements[6].text

dicoCaracJoueur['taille'] = float(re.sub('(\(|\)|m)','',re.search('(\([1-2]\.[0-9]{2}m\))',textTaille).group(1))) if textTaille else np.nan
dicoCaracJoueur['poids'] = float(re.sub('(\(|\)|kg)','',re.search('(\([0-9]{1,3}kg\))',textPoids).group(1))) if textPoids else np.nan
dicoCaracJoueur['pays'] = textPays
dicoCaracJoueur['date_naissance'] = pd.to_datetime(textDateNaissance)
dicoCaracJoueur['draft'] = textDraft

In [23]:
[e.text for e in nomPosition]

TypeError: 'WebElement' object is not iterable

In [20]:
[e.text for e in elements]

['7\'2" (2.18m)',
 '225lb (102kg)',
 'USA',
 'UCLA',
 'April 16, 1947',
 '1969 R1 Pick 1',
 '20 Years',
 '',
 '',
 '',
 '']

In [28]:
listeCaracNomPosition[1:3]

['Precious', 'Achiuwa']

In [26]:
listeCaracNomPosition

['Toronto Raptors | #5 | Forward', 'Precious', 'Achiuwa']

In [33]:
joueurs = JoueursSiteNba(2022, historiqueComplet=True, typeExport='Mano', liensMano=['https://www.nba.com/stats/player/76003/career'])

In [34]:
joueurs.dfJoueurs

,taille,poids,pays,date_naissance,draft,experience,nom,nom_simple,id_position_terrain,page_web
0,2.18,102.0,USA,1947-04-16,1969 R1 Pick 1,20 Years,Kareem Abdul-Jabbar,kareemabduljabbar,C,https://www.nba.com/stats/player/76003/career


In [35]:
joueurs = JoueursSiteNba(2022, historiqueComplet=True)

slider a cocher
element non trouve a adresse https://www.nba.com/player/1631230/dominick-barlow/
element non trouve a adresse https://www.nba.com/stats/player/76120/career/
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\martin.schoreisz\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\martin.schoreisz\AppData\Local\Temp\ipykernel_5596\4249355734.py", line 1, in <cell line: 1>
    joueurs = JoueursSiteNba(2022, historiqueComplet=True)
  File "C:\Users\martin.schoreisz\git\Basket\Basket\src\TelechargementDonnees.py", line 491, in __init__
    self.creerDfJoueur()
  File "C:\Users\martin.schoreisz\git\Basket\Basket\src\TelechargementDonnees.py", line 580, in creerDfJoueur
    self.dfJoueurs = pd.concat([pd.DataFrame(self.attributJoueur(l), index=[i]) for i,l in enumerate(self.listLinkJoueurs)], axis=0)
  File "C:\Users\martin.schoreisz\git\Basket\Basket\src\TelechargementDonnees.py", line 580, in <listcomp>
    self.dfJoueurs = pd.concat([pd.DataFrame(self.attributJoueur(l), index=[i]) for i,l in enumerate(self.listLi

In [49]:
joueurs.dfJoueurs.to_csv(r'D:\basket\dfJoueursBrutes.csv')

In [53]:
joueurs2 = JoueursSiteNba(2022, typeExport='Mano', historiqueComplet=True, 
                          liensMano=('https://www.nba.com/stats/player/1629061/career/', 'https://www.nba.com/stats/player/76048/career/','https://www.nba.com/player/1631317/adonis-arms/',
                                     'https://www.nba.com/player/1631230/dominick-barlow/', 'https://www.nba.com/stats/player/76548/career/', 'https://www.nba.com/player/1628368/deaaron-fox/',
                                     'https://www.nba.com/player/1628436/luke-kornet/', 'https://www.nba.com/player/1629641/romeo-langford/',
                                     'https://www.nba.com/stats/player/78041/career/', 'https://www.nba.com/stats/player/2351/career/'))

element non trouve a adresse https://www.nba.com/player/1631317/adonis-arms/
element non trouve a adresse https://www.nba.com/player/1631230/dominick-barlow/


In [55]:
joueurs2.dfJoueurs

,taille,poids,pays,date_naissance,draft,experience,nom,nom_simple,id_position_terrain
0,2.01,91.0,South Sudan,1997-02-01,2018,2 Years,Deng Adel,dengadel,F
1,2.01,100.0,USA,1946-01-11,1969 R2 Pick 25,1 Year,Wally Anderzunas,wallyanderzunas,C
2,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,USA,1905-04-07,Undrafted,1 Year,Red Dehnert,reddehnert,NC
5,1.91,84.0,USA,1997-12-20,2017 R1 Pick 5,5 Years,De'Aaron Fox,deaaronfox,G
6,2.18,113.0,USA,1995-07-15,Undrafted,5 Years,Luke Kornet,lukekornet,C-F
7,1.96,98.0,USA,1999-10-25,2019 R1 Pick 14,3 Years,Romeo Langford,romeolangford,G-F
8,2.08,109.0,USA,1958-12-16,1980 R2 Pick 25,8 Years,Jeff Ruland,jeffruland,C
9,1.88,88.0,USA,1924-09-11,Undrafted,1 Year,Bob Shea,bobshea,F


In [57]:
joueurs3 = JoueursSiteNba(2022, typeExport='Mano', historiqueComplet=True, 
                          liensMano=('https://www.nba.com/stats/player/77824/career/', 'https://www.nba.com/stats/player/78130/career/'))

In [58]:
joueurs3.dfJoueurs

,taille,poids,pays,date_naissance,draft,experience,nom,nom_simple,id_position_terrain
0,1.93,84.0,USA,1926-07-16,1951 R2 Pick 18,2 Years,Mel Payton,melpayton,G
1,1.88,88.0,USA,1924-09-11,Undrafted,1 Year,Bob Shea,bobshea,F


In [60]:
pd.concat([joueurs.dfJoueurs, joueurs2.dfJoueurs,joueurs3.dfJoueurs]).to_csv(r'D:\basket\dfJoueursBrutes.csv')

In [6]:
dfJoueursTotalImport = pd.read_csv(r'D:\basket\dfJoueursBrutes.csv')

In [12]:
dfJoueursTotalImport.loc[dfJoueursTotalImport.nom_simple == 'hankbeenders']

,Unnamed: 0,taille,poids,pays,date_naissance,draft,experience,nom,nom_simple,id_position_terrain
287,287,1.96,84.0,USA,1916-06-02,Undrafted,3 Years,Hank Beenders,hankbeenders,C


In [11]:
dfJoueursTotalImport.loc[dfJoueursTotalImport.duplicated('nom_simple', keep=False)]

,Unnamed: 0,taille,poids,pays,date_naissance,draft,experience,nom,nom_simple,id_position_terrain
2,2,2.18,102.0,USA,1947-04-16,1969 R1 Pick 1,20 Years,th,th,C
25,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,66,1.96,93.0,USA,1975-07-20,1996 R1 Pick 5,18 Years,th,th,G
105,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,111,2.11,110.0,Greece,1994-12-06,2013 R1 Pick 15,9 Years,th,th,F
114,114,2.01,108.0,USA,1984-05-29,2003 R1 Pick 3,19 Years,th,th,F
123,123,1.85,68.0,USA,1948-09-02,1970 R2 Pick 19,13 Years,th,th,G
129,129,1.93,86.0,USA,1928-04-09,1950,10 Years,th,th,F
131,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,215,1.98,114.0,USA,1963-02-20,1984 R1 Pick 5,16 Years,th,th,F


In [13]:
dfJoueursTotalImport

,Unnamed: 0,taille,poids,pays,date_naissance,draft,experience,nom,nom_simple,id_position_terrain
0,0,2.08,109.0,USA,1968-06-24,1990 R1 Pick 25,5 Years,Alaa Abdelnaby,alaaabdelnaby,F
1,1,2.06,107.0,USA,1946-04-07,1968 R1 Pick 5,10 Years,Zaid Abdul-Aziz,zaidabdulaziz,C
2,2,2.18,102.0,USA,1947-04-16,1969 R1 Pick 1,20 Years,th,th,C
3,3,1.85,73.0,USA,1969-03-09,1990 R1 Pick 3,9 Years,Mahmoud Abdul-Rauf,mahmoudabdulrauf,G
4,4,1.98,107.0,France,1974-11-03,1997 R1 Pick 11,7 Years,Tariq Abdul-Wahad,tariqabdulwahad,F-G
...,...,...,...,...,...,...,...,...,...,...
4820,8,2.08,109.0,USA,1958-12-16,1980 R2 Pick 25,8 Years,Jeff Ruland,jeffruland,C
4821,9,1.88,88.0,USA,1924-09-11,Undrafted,1 Year,Bob Shea,bobshea,F
4822,10,1.98,95.0,Venezuela,1976-12-18,Undrafted,2 Years,Oscar Torres,oscartorres,G-F
4823,0,1.93,84.0,USA,1926-07-16,1951 R2 Pick 18,2 Years,Mel Payton,melpayton,G
